In [1]:
#소리 파일을 다루기 위해 필요한 모듈
import numpy as np
import scipy.io.wavfile
import sounddevice as sd
import numpy as np
import librosa
import soundfile as sf

In [2]:
def open_wav_with_resample(file_name, bit, origin_rate, new_rate):
    data, rate = sf.read(file_name, dtype=bit)
    data = data.T
    data = librosa.resample(data, origin_rate, new_rate)
    return data, new_rate

def open_wav(file_name, origin_rate):
    data, rate = librosa.load(file_name, sr=origin_rate)
    return data, rate

def open_wav(file_name, origin_rate):
    data, rate = librosa.load(file_name, sr=origin_rate)
    return data, rate

def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[start:end]

#다수의 파일을 10초 간격으로 자른다. 4초 이하의 파일은 생성하지 않는다.
#입력 파일은 result+i(숫자)의 형식의 wav 파일이어야한다.
def Execute_split(start, end):
    num_of_file = 0
    for n_file in range(start,end+1):
        print("now splitting",str(n_file)+"th file..")
        file_name = 'result'+str(n_file)+'.wav'
        sample_rate = 192000 # 192KHz
        data_length = sample_rate * 60 # 192KHz * 60
        wav_data, sample_rate = open_wav(file_name, sample_rate)
        sec = int(wav_data.shape[0]/192000)
        rest = sec%10
        for split_time in range(0,sec-rest,10):
            num_of_file += 1
            edited_data = split_wav(wav_data,sample_rate,split_time,split_time+10)
            result_file_name = 'split'+str(num_of_file)+'.wav'
            sample_format = 'PCM_16' # PCM_24, PCM_16 PCM_s8 등 옵션있음.
            sf.write(result_file_name, edited_data, sample_rate, subtype=sample_format)
        num_of_file += 1
        # 모델에 너무 짧은 음성은 학습시키지 않는다.
        if rest > 4:
            edited_data = split_wav(wav_data,sample_rate,sec-rest,sec)
            result_file_name = 'split'+str(num_of_file)+'.wav'
            sample_format = 'PCM_16' # PCM_24, PCM_16 PCM_s8 등 옵션있음.
            sf.write(result_file_name, edited_data, sample_rate, subtype=sample_format)

def edit_one_file(file_name,result_file_name='result',start_time=0,end_time=10):
    sample_rate = 192000 # 192KHz
    data_length = sample_rate * 60 # 192KHz * 60
    wav_data, sample_rate = open_wav(file_name, sample_rate)
    edited_data = split_wav(wav_data,sample_rate,start_time,end_time)
    sample_format = 'PCM_16' # PCM_24, PCM_16 PCM_s8 등 옵션있음.
    sf.write(result_file_name, edited_data, sample_rate, subtype=sample_format)


In [41]:
Execute_split(1,7)

now splitting 1th file..
now splitting 2th file..
now splitting 3th file..
now splitting 4th file..
now splitting 5th file..
now splitting 6th file..
now splitting 7th file..


In [6]:
edit_one_file("ssk_vocals.wav","result.wav",13,120)